# Chapter 8.5 - Introduction to generative adversarial networks

In [1]:
import numpy as np
from keras.layers import Dense, LeakyReLU, Input, Conv2DTranspose, Reshape, Conv2D
from keras.models import Model

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
latent_dim = 32
height = 32
width = 32
channels = 3

## The generator

In [3]:
generator_input = Input(shape=(latent_dim,))

# First, transform the input into a 16x16 128-channels feature map
x = Dense(units = 128 * 16 * 16)(generator_input)
x = LeakyReLU()(x)
x = Reshape(target_shape = (16, 16, 128))(x)

# Then, add a convolution layer
x = Conv2D(filters = 256, 
           kernel_size = (5, 5), 
           padding = 'same')(x)
x = LeakyReLU()(x)

# Upsample to 32x32
x = Conv2DTranspose(filters = 256, 
                    kernel_size = (4, 4), 
                    strides = (2, 2), 
                    padding = 'same')(x)
x = LeakyReLU()(x)

# Few more conv layers
x = Conv2D(filters = 256, 
           kernel_size = (5, 5),  
           padding = 'same')(x)
x = LeakyReLU()(x)
x = Conv2D(filters = 256, 
           kernel_size = (5, 5), 
           padding = 'same')(x)
x = LeakyReLU()(x)

# Produce a 32x32 1-channel feature map
x = Conv2D(filters = channels, 
           kernel_size = (7, 7),
           activation = 'tanh', 
           padding = 'same')(x)
generator = Model(generator_input, x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

## The discriminator

In [4]:
from keras.layers import Flatten, Dropout

In [5]:
discriminator_input = Input(shape = (height, width, channels))
x = Conv2D(filters = 128,  
           kernel_size = (3, 3))(discriminator_input)
x = LeakyReLU()(x)
x = Conv2D(filters = 128,  
           kernel_size = (4, 4), 
           strides = (2, 2))(x)
x = LeakyReLU()(x)
x = Conv2D(filters = 128,  
           kernel_size = (4, 4), 
           strides = (2, 2))(x)
x = LeakyReLU()(x)
x = Conv2D(filters = 128,  
           kernel_size = (4, 4), 
           strides = (2, 2))(x)
x = LeakyReLU()(x)
x = Flatten()(x)

# One dropout layer - important trick!
x = Dropout(rate = 0.4)(x)

# Classification layer
x = Dense(units = 1, 
          activation = 'sigmoid')(x)

discriminator = Model(discriminator_input, x)
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

In [6]:
from keras.optimizers import RMSprop

In [7]:
# To stabilize training, we use learning rate decay
# and gradient clipping (by value) in the optimizer.
discriminator_optimizer = RMSprop(lr = 0.0008, 
                                  clipvalue = 1.0, 
                                  decay = 1e-8)
discriminator.compile(optimizer = discriminator_optimizer, 
                      loss = 'binary_crossentropy')

## The adversarial network

In [8]:
# Set discriminator weights to non-trainable
# (will only apply to the `gan` model)
discriminator.trainable = False

gan_input = Input(shape = (latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)

gan_optimizer = RMSprop(lr = 0.0004, 
                        clipvalue = 1.0, 
                        decay = 1e-8)
gan.compile(optimizer = gan_optimizer, 
            loss = 'binary_crossentropy')

# Training a DCGAN

In [9]:
import os
from keras.preprocessing import image
from keras.datasets import cifar10

In [10]:
# Load CIFAR10 data
(x_train, y_train), (_, _) = cifar10.load_data()

# Select images (class 5)
x_train = x_train[y_train.flatten() == 5]

In [11]:
x_train.shape

(5000, 32, 32, 3)

In [12]:
# Normalize data
x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20

In [13]:
x_train.shape

(5000, 32, 32, 3)

In [14]:
save_dir = './data/Chapter 8.5 - Introduction to generative adversarial networks/'

In [15]:
# Start training loop
start = 0
for step in range(iterations):
    # Sample random points in the latent space
    random_latent_vectors = np.random.normal(size = (batch_size, latent_dim))

    # Decode them to fake images
    generated_images = generator.predict(random_latent_vectors)

    # Combine them with real images
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # Assemble labels discriminating real from fake images
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # Add random noise to the labels - important trick!
    labels = labels + 0.05 * np.random.random(labels.shape)

    # Train the discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # Sample random points in the latent space
    random_latent_vectors = np.random.normal(size = (batch_size, latent_dim))

    # Assemble labels that say "all real images"
    misleading_targets = np.zeros((batch_size, 1))

    # Train the generator (via the gan model,
    # where the discriminator weights are frozen)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start = start + batch_size
    if start > len(x_train) - batch_size:
        start = 0

    # Occasionally save / plot
    if step % 100 == 0:
        # Save model weights
        gan.save_weights('./saved_checkpoints/Chapter 8.5 - Introduction to generative adversarial networks/gan.h5')

        # Print metrics
        print('-' * 50)
        print('Step: %s' % step )
        print('Discriminator loss: %s' % d_loss)
        print('Adversarial loss: %s' % a_loss)

        # Save one generated image
        img = image.array_to_img(generated_images[0] * 255., 
                                 scale = False)
        img.save(os.path.join(save_dir, 'generated_dog' + str(step) + '.png'))

        # Save one real image, for comparison
        img = image.array_to_img(real_images[0] * 255., 
                                 scale = False)
        img.save(os.path.join(save_dir, 'real_dog' + str(step) + '.png'))

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:953: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


--------------------------------------------------
Step: 0
Discriminator loss: 0.6911715
Adversarial loss: 0.6933484
--------------------------------------------------
Step: 100
Discriminator loss: 0.44683647
Adversarial loss: 2.628831
--------------------------------------------------
Step: 200
Discriminator loss: 0.70925105
Adversarial loss: 0.8020114
--------------------------------------------------
Step: 300
Discriminator loss: 0.7145446
Adversarial loss: 0.77895004
--------------------------------------------------
Step: 400
Discriminator loss: 0.7032187
Adversarial loss: 0.7404046
--------------------------------------------------
Step: 500
Discriminator loss: 0.69979143
Adversarial loss: 0.7711795
--------------------------------------------------
Step: 600
Discriminator loss: 0.6790079
Adversarial loss: 1.15787
--------------------------------------------------
Step: 700
Discriminator loss: 0.69431365
Adversarial loss: 0.7624009
------------------------------------------------

--------------------------------------------------
Step: 6700
Discriminator loss: 0.71096146
Adversarial loss: 0.7600935
--------------------------------------------------
Step: 6800
Discriminator loss: 0.6972765
Adversarial loss: 0.7767426
--------------------------------------------------
Step: 6900
Discriminator loss: 0.68509376
Adversarial loss: 0.8056838
--------------------------------------------------
Step: 7000
Discriminator loss: 0.7230104
Adversarial loss: 0.7839001
--------------------------------------------------
Step: 7100
Discriminator loss: 0.6922885
Adversarial loss: 0.75965863
--------------------------------------------------
Step: 7200
Discriminator loss: 0.6749347
Adversarial loss: 0.6775611
--------------------------------------------------
Step: 7300
Discriminator loss: 0.7141993
Adversarial loss: 0.7120148
--------------------------------------------------
Step: 7400
Discriminator loss: 0.68293697
Adversarial loss: 0.7780585
------------------------------------